In [1]:
with open("requirements.txt", "w") as f:
    f.write("kfp==1.8.9\n")
    f.write("requests\n")
    
!pip install -r requirements.txt  --upgrade --user

     |████████████████████████████████| 62 kB 306 kB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
import kfp.dsl as dsl
import kfp
from kfp import components

kfserving_op = components.load_component_from_url('https://raw.githubusercontent.com/kubeflow/pipelines/master/components/kubeflow/kfserving/component.yaml')

@dsl.pipeline(
  name='KFServing pipeline',
  description='A pipeline for KFServing.'
)
def kfservingPipeline(
    action='apply',
    model_name='tensorflow-sample',
    model_uri='gs://kfserving-examples/models/tensorflow/flowers',
    namespace='kubeflow-user-example-com',
    framework='tensorflow'):

    # flowers model was trained with 5 category: [daisy(雛菊)  dandelion(浦公英)  roses(玫瑰)  sunflowers(向日葵)  tulips(鬱金香)]
    # see details here:  https://github.com/tensorflow/hub/blob/master/examples/colab/image_feature_vector.ipynb
    
    kfserving = kfserving_op(action = action,
                             model_name=model_name,
                             model_uri=model_uri,
                             namespace=namespace,
                             framework=framework).set_image_pull_policy('Always')

In [3]:
# Compile pipeline
kfp.compiler.Compiler().compile(kfservingPipeline, 'tf-flower.zip')

Please upload the manifests generated above(tf-flower.zip)
And Below is example scripts to send requests to the model you deployed
Or you can run this on your host machines under hack/hack/local-test-with-kfserving.py

##### this is fixed in v1.3.1, so you are no longer to take actions
~note: please disable sidecar-injection on kubeflow-user namespace,
as knative-serving / knative-events are not enabled istio sidecar.
failed to disable could get RBAC error~


##### you have to REPLACE authervice_session token into yours. Go to Browser -> Developer Console -> Application -> Cookies
![title](img/cookies.png)

In [5]:
## the following example use python's request to send restapi requests
import base64
import json
import requests

with open('floribunda.jpg', 'rb') as f:
    image_content = f.read()
    image_64_encode = base64.encodebytes(image_content).decode('utf-8')
headers = {"Cookie": "authservice_session=MTYzODQzMjc4OXxOd3dBTkRkWVZsZE5RMFV6TlU5YVREWkxWa1JKUVZKVE1razFSVFV6TTFwVVdVbFdSVkZLVmxsS1VsUk5Xa0l5U2tkTFMwNUNSVkU9fD4yeVFk3-edCElo-COm7DWyuMOjkN0vrbSSRTLaxeJT",
          "Host": "tensorflow-sample.kubeflow-user-example-com.example.com"}
payload = {"instances":[{"image_bytes": {"b64": image_64_encode}, "key": "1"}]}
resp = requests.post('http://istio-ingressgateway.istio-system/v1/models/tensorflow-sample:predict', headers=headers, data=json.dumps(payload))
print(resp.text)

{
    "predictions": [
        {
            "scores": [1.30671893e-07, 3.01086693e-08, 0.814807534, 9.6436537e-08, 0.185192183, 3.43902293e-08],
            "prediction": 2,
            "key": "1"
        }
    ]
}


In [ ]:
## the following example use curl to send restapi requests

!(echo -n '{"instances":[{"image_bytes": {"b64": "'; base64 sunflowers.jpg; echo '"}, "key": "1"}]}') | \
curl -XPOST -vvv -H "Host: tensorflow-sample.kubeflow-user-example-com.example.com" \
-H "Cookie: authservice_session=MTYzMDg5MjU5NnxOd3dBTkZKS1RWRk1VRkZGVHpWQk5Fa3pNMHhPTXpkV1draFVSMWN6U3pKVFFrVlJUMU5DU0ZoR1JGcFZXVFZaUmxnMFF6Tk9TVUU9fId9HVT3LD8wbAnYPmQ7v2k1-yLfGzNMBbAxKkcWh48-" \
-d @- \
http://istio-ingressgateway.istio-system/v1/models/tensorflow-sample:predict